## PRUEBA 2

In [ ]:
!pip install scikit-learn
!pip install seaborn
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 122.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have te

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import shutil
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

In [ ]:
# Definir ruta base de datos con subcarpetas "benigno" y "maligno"
base_dir = "/content/drive/MyDrive/ulstrasound images"

In [ ]:
# Crear nuevas rutas para la separación automática
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

# Parámetros de preprocesamiento y entrenamiento
IMG_SIZE = (300, 300)
BATCH_SIZE = 32
EPOCHS = 60

# Callbacks para optimización
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Generadores de imágenes con Data Augmentation refinado
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

val_test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


In [ ]:
# Cargar modelo EfficientNetB3 preentrenado con ajuste óptimo de capas
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_tensor=Input(shape=(300, 300, 3)))
for layer in base_model.layers[-40:]:  # Se descongelan 40 capas para mejorar aprendizaje
    layer.trainable = True

# Construcción del modelo con capas adicionales
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
out = Dense(1, activation='sigmoid')(x)  # Clasificación binaria

model = Model(inputs=base_model.input, outputs=out)

# Estrategia de Optimización con Cosine Decay
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.0001, decay_steps=EPOCHS * 100, alpha=0.00001)
optimizer = AdamW(learning_rate=lr_schedule)

model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Mostrar resumen del modelo
model.summary()

In [ ]:
import gc
tf.keras.backend.clear_session()
gc.collect()

In [ ]:
# Entrenamiento del modelo con callbacks
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=60,
    callbacks=[early_stopping]
)

In [ ]:
# Evaluación del modelo en el conjunto de prueba
results = model.evaluate(test_generator)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")

In [ ]:
# Plot training curves
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Loss
    ax1.plot(history.history['loss'], label='Training')
    ax1.plot(history.history['val_loss'], label='Validation')
    ax1.set_title('Loss During Training')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()

    # Accuracy
    ax2.plot(history.history['accuracy'], label='Training')
    ax2.plot(history.history['val_accuracy'], label='Validation')
    ax2.set_title('Accuracy During Training')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.legend()

    plt.show()

# Show metrics
plot_training_history(history)

In [ ]:
def plot_results(model, test_generator):
    # Get predictions from the model
    y_pred_probs = model.predict(test_generator)
    y_pred = (y_pred_probs > 0.5).astype("int32")  # Convert to 0 or 1

    # Get true labels
    y_true = test_generator.classes

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Benign", "Malignant"], yticklabels=["Benign", "Malignant"])
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()

    # Classification report
    print("Classification Report:\n", classification_report(y_true, y_pred, target_names=["Benign", "Malignant"]))

    # ROC curve
    fpr, tpr, _ = roc_curve(y_true, y_pred_probs)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(6, 6))
    plt.plot(fpr, tpr, color="blue", label=f"Area under the curve (AUC) = {roc_auc:.2f}")
    plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend(loc="lower right")
    plt.show()

# Call the function after evaluating the model
plot_results(model, test_generator)

In [ ]:
# Guardar el modelo en un archivo .h5
model.save('EfficientNetB3.h5')